# 1. 소상공인시장 데이터 불러오기

In [244]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import os

In [19]:
market_data = pd.read_csv('market_data.csv', encoding='utf-8', sep='|')
market_data.head()

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,17163092,도전최강달인왕만두,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1174010200102170000018014,고덕그라시움,서울특별시 강동구 고덕로 333,134082,5224.0,NaN,NaN,NaN,127.159471,37.556197
1,17120456,이때,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1144012400103900067027687,NaN,서울특별시 마포구 동교로38안길 7,121867,3982.0,NaN,NaN,NaN,126.924660,37.562176
2,17175350,L.A.D,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1144012000103950112010755,NaN,서울특별시 마포구 잔다리로3안길 23,121840,4043.0,NaN,NaN,NaN,126.919845,37.550689
3,17175311,제이씨에스푸드,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1162010200101180033018722,NaN,서울특별시 관악구 신림로14길 3,151856,8839.0,NaN,NaN,NaN,126.937790,37.471190
4,22767534,BYC상신점,상신점,D,소매,D05,의복의류,D05A07,셔츠/내의/속옷,NaN,...,1171011300100360000022458,성암빌딩,서울특별시 송파구 오금로 527,138110,5768.0,NaN,1,NaN,127.147321,37.493054


데이터가 잘 분류된 것을 확인할 수 있었다.
카페만 따로 떼서 볼 수 있을까?

In [20]:
cafe_data = market_data[market_data["상권업종중분류명"] == '커피점/카페']

In [21]:
cafe_data["상호명"]

37              커피빈
129           요거프레소
280        406번째스토브
281           로얄커피숍
297          빈트리망원점
            ...    
346537    로컬스티치북앤커피
346538         카페시모
346539         만랩커피
346540        레프트커피
346541        더쌍화커피
Name: 상호명, Length: 22239, dtype: object

# 2. 크롤러 기본 동작

In [1]:
from pathlib import Path

In [2]:
driverPath = Path(r"/Users/jun/My Works/utils/chromedriver")

In [3]:
print(driverPath)

/Users/jun/My Works/utils/chromedriver


In [13]:
import selenium
from selenium import webdriver

In [17]:
with webdriver.Chrome(driverPath) as driver:
    driver.get('https://www.diningcode.com/')
    driver.implicitly_wait(3)

In [3]:
names = pd.read_csv('seoul names.csv', index_col=0)
names

,0
Unnamed: 0,
0,둔촌2동
1,둔촌1동
2,길동
3,성내3동
4,성내2동
...,...
418,평창동
419,부암동
420,삼청동


In [197]:
driver = webdriver.Chrome(driverPath)

In [191]:
base_url = "https://www.diningcode.com/2018/ajax/list.php"

In [192]:
names.iloc[0].item()

'둔촌2동'

In [193]:
page = 1

In [194]:
url = base_url + '?' + 'query=' + names.iloc[0].item() + '카페' + '&' + 'page=' + str(page)

In [198]:
driver.get(url)

In [199]:
html = driver.page_source

In [200]:
soup = BeautifulSoup(html, 'html.parser')

In [201]:
soup.find_all("span", class_="btxt")[0].text.split(' ', 1)[1] # span 태그 btxt를 찾아서 i번째 원소의 텍스트를 추출하고 첫 번째 ' '에서 잘라(번호 제거) 저장

'엘리펀트커피'

In [202]:
soup.find_all("span", class_="review button")

[<span class="review button" onclick="getReview('kZDSMy8d7k1O');">2</span>,
 <span class="review button" onclick="getReview('pfEZzZ1KBNju');">4</span>,
 <span class="review button" onclick="getReview('OuVvge8e89rJ');">2</span>,
 <span class="review button" onclick="getReview('5h79CutHiwLz');">1</span>,
 <span class="review button" onclick="getReview('0CdvoPI1DxDU');">1</span>,
 <span class="review button" onclick="getReview('5e9wVZmdxQ2L');">0</span>,
 <span class="review button" onclick="getReview('RZr5rLdepctH');">1</span>,
 <span class="review button" onclick="getReview('8MM6o27sXlay');">0</span>,
 <span class="review button" onclick="getReview('V7GIu2tZnAN8');">0</span>,
 <span class="review button" onclick="getReview('Qjlu7CDgARf9');">0</span>]

In [205]:
cafe_list = [data.text.split(' ', 1)[1] for data in soup.find_all("span", class_="btxt")]
cafe_link = [data.get("href") for data in soup.find_all("a", class_='blink')]
review_counts = [int(data.text) for data in soup.find_all("span", class_="review button")]

In [101]:
len(cafe_list)

10

In [206]:
review_counts

[2, 4, 2, 1, 1, 0, 1, 0, 0, 0]

한 번에 다 수행하는 크롤러 동작! (카페의 이름을 가져오는 방향)

In [1]:
base_url = "https://www.diningcode.com/2018/ajax/list.php"
total_cafe = []

with webdriver.Chrome(driverPath) as driver:
    # 각 카페의 이름을 가져옵니다.
    for index, data in names[:10].iterrows():
        page = 1
        while(True):
            url = base_url + '?' + 'query=' + names.iloc[0].item() + '카페' + '&' + 'page=' + str(page) # 주소 설정
            driver.get(url) # 크롬으로 페이지 접속
            html = driver.page_source # 페이지 소스 저장
            soup = BeautifulSoup(html, 'html.parser') # 스프로 파싱
            
            # 데이터에서 텍스트를 추출합니다
            cafe_list = [data.text.split(' ', 1)[1] for data in soup.find_all("span", class_="btxt")]
            cafe_link = [data.get("href") for data in soup.find_all("a", class_='blink')]
            review_counts = [int(data.text) for data in soup.find_all("span", class_="review button")]
            
            # 추출한 데이터를 묶어서 반복합니다.
            for pair in zip(cafe_list, cafe_link, review_counts):
                
                if pair[2] != 0: # 리뷰가 있는 카페만 크롤링합니다.
                    # 데이터 저장용 딕셔너리를 생성합니다
                    cafe_dict = {
                            'name': pair[0],
                            'link': pair[1]
                    }

                    # 만든 딕셔너리가 배열에 없으면 추가합니다.
                    if cafe_dict not in total_cafe:
                        total_cafe.append(cafe_dict)
            
            if len(cafe_list) != 10: break # 목록이 10개 미만이면 최대 개수에 도달했다는 뜻입니다.   
            page += 1
            
    

NameError: name 'webdriver' is not defined

In [208]:
total_cafe

[{'name': '엘리펀트커피', 'link': '/profile.php?rid=kZDSMy8d7k1O'},
 {'name': '드 까르멜릿', 'link': '/profile.php?rid=pfEZzZ1KBNju'},
 {'name': '전광수 커피하우스 천호동점', 'link': '/profile.php?rid=OuVvge8e89rJ'},
 {'name': '카페 원앤온리', 'link': '/profile.php?rid=5h79CutHiwLz'},
 {'name': '외계인커피', 'link': '/profile.php?rid=0CdvoPI1DxDU'},
 {'name': '바내', 'link': '/profile.php?rid=RZr5rLdepctH'}]

In [225]:
driver = webdriver.Chrome(driverPath)

In [223]:
url = "https://www.diningcode.com/profile.php?rid=5h79CutHiwLz"

In [226]:
driver.get(url)

In [227]:
driver.find_element_by_xpath("//*[@id='div_profile']/div[1]/ul/li[1]").click()

In [228]:
driver.find_element_by_xpath("//*[@id='full-area']/div[6]/span[2]").click()

In [213]:
img = driver.find_element_by_id('display-img')
src = img.get_attribute('src')

In [214]:
urllib.request.urlretrieve(src, "image.jpg")

('image.jpg', <http.client.HTTPMessage at 0x7ffa04749310>)

In [233]:
driver.find_element_by_class_name('btn-gallery-next').click()

In [234]:
driver.close()

이미지를 저장하는 크롤러를 만듭니다.

In [248]:
# 카페 상세정보 link는 uri가 포함되어있습니다.
base_url = "https://www.diningcode.com/"
src = ""

with webdriver.Chrome(driverPath) as driver:
    for cafe in total_cafe[:1]:

        # url 생성 후 접근
        url = base_url + cafe['link']
        driver.get(url)
        
        # 첫 번째 사진 클릭 (사진 모드로 들어갑니다)
        driver.find_element_by_xpath("//*[@id='div_profile']/div[1]/ul/li[1]").click()
        driver.implicitly_wait(3) # 버튼이 등장할때까지 기다립니다.
        # 음식/실내외 버튼 클릭
        driver.find_element_by_xpath("//*[@id='full-area']/div[6]/span[2]").click()
        
        # 3장을 가져와야 하므로 3번 반복합니다
        for i in range(3):
            img = driver.find_element_by_id('display-img')
            
            if src == img.get_attribute('src') : break # 이미지가 3장 미만인경우, 버튼 클릭이 안 돼서 사진이 넘어가지 않습니다. 앞과 같은 사진이면 반복을 중지합니다.
                
            # 앞과 다른 사진이면 src 변수를 업데이트합니다.
            src = img.get_attribute('src')
            # 카페이름/image[번호].jpg 로 저장합니다.
            
            dirName = Path("images/" + cafe['name'])
            
            if not os.path.isdir(dirName):
                os.makedirs(dirName)
            urllib.request.urlretrieve(src, os.path.join(dirName, str(i+1) + ".jpg"))
            
            # 다음 버튼을 클릭합니다.
            driver.find_element_by_class_name('btn-gallery-next').click()
            driver.implicitly_wait(1)

# 전체 크롤러
파이썬 모듈화용 크롤러 입니다.

In [5]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import os
from pathlib import Path
import selenium
from selenium import webdriver
# 예외처리 위한 라이브러리
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException

driverPath = Path(r"/Users/jun/My Works/utils/chromedriver") # 드라이버 경로 설정
names = pd.read_csv('seoul names.csv', index_col=0) # 서울시 행정구역 정보 불러오기

base_url = "https://www.diningcode.com/2018/ajax/list.php"
total_cafe = []
name_list = [] 
main_tag_list = []
score_list = []
star_score = []
like = []
locate = []
taste_score = []
price_score = []
service_score = []
feat_text=[]


with webdriver.Chrome(driverPath) as driver:
    driver.implicitly_wait(5) # 설명을 위한 대기
    # ---------카페 이름, 링크 크롤링 ----------------
    # 각 카페의 이름을 가져옵니다.
    for index, data in names[:10].iterrows():
        page = 1 # 한 페이지에 10개씩 보여줍니다.
        while(True):
            url = base_url + '?' + 'query=' + names.iloc[0].item() + '카페' + '&' + 'page=' + str(page) # 주소 설정
            driver.get(url) # 크롬으로 페이지 접속
            
            
            html = driver.page_source # 페이지 소스 저장
            soup = BeautifulSoup(html, 'html.parser') # 스프로 파싱
            
            # 데이터에서 텍스트를 추출합니다
            cafe_list = [data.text.split(' ', 1)[1] for data in soup.find_all("span", class_="btxt")]
            cafe_link = [data.get("href") for data in soup.find_all("a", class_='blink')]
            review_counts = [int(data.text) for data in soup.find_all("span", class_="review button")]
            
            # 추출한 데이터를 묶어서 반복합니다.
            for pair in zip(cafe_list, cafe_link, review_counts):
                
                if pair[2] != 0: # 리뷰가 있는 카페만 크롤링합니다.
                    # 데이터 저장용 딕셔너리를 생성합니다
                    cafe_dict = {
                            'name': pair[0],
                            'link': pair[1]
                    }

                    # 만든 딕셔너리가 배열에 없으면 추가합니다.
                    if cafe_dict not in total_cafe:
                        total_cafe.append(cafe_dict)
            
            if len(cafe_list) != 10: break # 목록이 10개 미만이면 최대 개수에 도달했다는 뜻입니다.   
            page += 1
            driver.implicitly_wait(10)
            
    all_cafe = pd.DataFrame(total_cafe)
    all_cafe.to_csv('all_cafe.csv')
    
    # --------- 이미지 크롤링 --------
    # 카페 상세정보 link는 uri가 포함되어있습니다.
    base_url = "https://www.diningcode.com/"
    src = ""
    
    # 위에서 생성된 total_cafe 배열을 사용해 접근합니다.
    for cafe in total_cafe[:2]:

        # url 생성 후 접근
        url = base_url + cafe['link']
        driver.get(url)
        
        # 첫 번째 사진 클릭 (사진 모드로 들어갑니다)
        driver.find_element_by_xpath("//*[@id='div_profile']/div[1]/ul/li[1]").click()
        driver.implicitly_wait(10) # 버튼이 등장할때까지 기다립니다.
        # 음식/실내외 버튼 클릭
        driver.find_element_by_xpath("//*[@id='full-area']/div[6]/span[2]").click()
        
        # 3장을 가져와야 하므로 3번 반복합니다
        for i in range(3):
            img = driver.find_element_by_id('display-img')
            
            if src == img.get_attribute('src') : break # 이미지가 3장 미만인경우, 버튼 클릭이 안 돼서 사진이 넘어가지 않습니다. 앞과 같은 사진이면 반복을 중지합니다.
                
            # 앞과 다른 사진이면 src 변수를 업데이트합니다.
            src = img.get_attribute('src')
            # 카페이름/image[번호].jpg 로 저장합니다.
            
            dirName = Path("images/" + cafe['name'])
            
            if not os.path.isdir(dirName):
                os.makedirs(dirName)
            urllib.request.urlretrieve(src, os.path.join(dirName, str(i+1) + ".jpg"))
            
            # 다음 버튼을 클릭합니다.
            driver.find_element_by_class_name('btn-gallery-next').click()
            driver.implicitly_wait(1)
    
    
        # url 생성 후 접근
        url = base_url + cafe['link']
        driver.get(url)
        try:
            # 카페명
            name_list.append(driver.find_element_by_xpath("//*[@id='div_profile']/div[1]/div[2]/p").text)
        except NoSuchElementException:
            name_list.append('-')
        try:
            #메인태그
            main_tag_list.append(driver.find_element_by_xpath("//*[@id='div_profile']/div[2]/ul/li[3]/span").text)    
        except NoSuchElementException:
            main_tag_list.append('-')
        try:
            # 스코어
            score_list.append(driver.find_element_by_xpath("//*[@id='div_profile']/div[1]/div[4]/p/strong").text)
        except NoSuchElementException:
            score_list.append('-')
        try:
            # 별점 5점 만점
            star_score.append(driver.find_element_by_xpath("//*[@id='lbl_review_point']").text)
        except NoSuchElementException:
            star_score_list.append('-')

        try:
            # 좋아요수
            like.append(driver.find_element_by_xpath("//*[@id='div_profile']/div[1]/div[5]/a[1]/span/i").text)
        except NoSuchElementException:
            like_list.append('-')
        try:
            # 주소
            locate.append(driver.find_element_by_xpath("//*[@id='div_profile']/div[2]/ul/li[1]").text)
        except NoSuchElementException:
            locate_list.append
        try:
            # 맛 스코어
            taste_score.append(driver.find_element_by_xpath("//*[@id='div_profile']/div[8]/div[1]/p[2]/span[1]/i").text)
        except NoSuchElementException:
            taste_score.append('-')
        try:
            # 가격 스코어
            price_score.append(driver.find_element_by_xpath("//*[@id='div_profile']/div[8]/div[1]/p[2]/span[2]/i").text)
        except NoSuchElementException:
            price_score.append('-')
        try:
            # 서비스 스코어
            service_score.append(driver.find_element_by_xpath("//*[@id='div_profile']/div[8]/div[1]/p[2]/span[3]/i").text)
        except NoSuchElementException:
            service_score.append('-')
        try:
            # 특징
            feat_text.append(driver.find_element_by_xpath("//*[@id='div_profile']/div[2]/ul/li[4]").text)
        except NoSuchElementException:
            feat_text.append('-')

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=90.0.4430.93)


In [2]:
crawling_data = pd.DataFrame([name_list,main_tag_list,score_list,star_score,like,locate,taste_score,price_score,service_score,feat_text]).T
crawling_data.columns = ['name','main_tag','score','star_score','like','locate','taste_score','price_score','service_score','feat_text']
crawling_data

,name,main_tag,score,star_score,like,locate,taste_score,price_score,service_score,feat_text
0,엘리펀트커피,혼카페,33점,4.5점,11,서울특별시 강동구 둔촌2동 53-3,5.0,2.9,5.0,"빈티지, 무료주차"
1,드 까르멜릿,돌잔치,24점,2.8점,42,서울특별시 강동구 둔촌동 213-1,0,0,0,"디너코스, 주차"


In [15]:
pd.read_csv("all_cafe", index_col=0).to_dict('records')

[{'name': '엘리펀트커피', 'link': '/profile.php?rid=kZDSMy8d7k1O'},
 {'name': '드 까르멜릿', 'link': '/profile.php?rid=pfEZzZ1KBNju'},
 {'name': '전광수 커피하우스 천호동점', 'link': '/profile.php?rid=OuVvge8e89rJ'},
 {'name': '카페 원앤온리', 'link': '/profile.php?rid=5h79CutHiwLz'},
 {'name': '외계인커피', 'link': '/profile.php?rid=0CdvoPI1DxDU'},
 {'name': '바내', 'link': '/profile.php?rid=RZr5rLdepctH'}]